## Imports

In [1]:
import cv2
import numpy as np
from glob import glob
import os

## Phase correlation alignment function

In [3]:
def phase_correlation_alignment(patch1, patch2):
    patch1_freq = np.fft.fftn(patch1)
    patch2_freq = np.fft.fftn(patch2)
    corr = np.fft.ifftn(patch1_freq*np.conjugate(patch2_freq)/(np.abs(patch1_freq)*np.abs(patch2_freq)))

    ty, tx = np.unravel_index(np.argmax(corr), corr.shape)  # this gives us the translation between patches
    if tx > patch1.shape[1] // 2:
        tx -= patch1.shape[1]

    if ty > patch1.shape[0] // 2:
        ty -= patch1.shape[0]

    rows, cols = patch2.shape
    M = np.float32([[1, 0, -tx],[0, 1, -ty]])
    aligned_patch = cv2.warpAffine(patch2, M, (cols,rows))
    return aligned_patch

## Imports and alignment

In [18]:
file_list = sorted(glob("D:/Datasets/MLDM Project/Data/0.20__2/cropped/*.jpg"))
#file_list = file_list[::-1]
base_image_path = file_list[0]
aligned_images = []

res = 900

def get_bounding_box(image):
    # Get the coordinates of non-black pixels
    coords = np.argwhere(image)
    # If there are no non-black pixels, return None
    if coords.size == 0:
        return None
    # Get the bounding box
    x_min, y_min = coords.min(axis=0)
    x_max, y_max = coords.max(axis=0)
    return x_min, y_min, x_max, y_max

def pad_image_to_size(image, target_size):
    # Pad the image to the target size
    pad_y = max(0, target_size - image.shape[0])
    pad_x = max(0, target_size - image.shape[1])
    padded_image = cv2.copyMakeBorder(image, 0, pad_y, 0, pad_x, cv2.BORDER_CONSTANT, value=0)
    return padded_image

def inpaint_black_regions(image):
    # Create a mask of the black regions
    mask = cv2.inRange(image, 0, 0)
    # Inpaint the black regions in the image
    inpainted_image = cv2.inpaint(image, mask, 3, cv2.INPAINT_TELEA)
    return inpainted_image

for i in range(0, len(file_list)):
    img_path = file_list[i]
    print("Processing image ", img_path)

    # Extract a 500x500 patch from the base image and the current image
    base_patch = cv2.imread(base_image_path, 0)[0:res, 0:res]
    img_patch = cv2.imread(img_path, 0)[0:res, 0:res]
    
    # Convert the patches to grayscale if they are color images
    if base_patch.ndim == 3:
        print("Converting base patch to grayscale")
        base_patch = cv2.cvtColor(base_patch, cv2.COLOR_BGR2GRAY)
    if img_patch.ndim == 3:
        print("Converting patch to grayscale")
        img_patch = cv2.cvtColor(img_patch, cv2.COLOR_BGR2GRAY)


    aligned_patch = phase_correlation_alignment(base_patch, img_patch)
    print("Patch aligned using phase correlation")

    # Skip if the aligned patch is entirely black
    if np.all(aligned_patch == 0):
        print("Skipping image due to failed alignment")
        continue

    # Get the bounding box of the non-black regions
    bbox = get_bounding_box(aligned_patch)
    if bbox is None:
        print("Skipping image due to no non-black pixels")
        continue

    x_min, y_min, x_max, y_max = bbox

    # Crop the aligned image
    cropped_patch = aligned_patch[x_min:x_max+1, y_min:y_max+1]

    # Pad the cropped patch to 500x500 if necessary
    cropped_patch = pad_image_to_size(cropped_patch, res)

    # Inpaint the black regions in the cropped patch
    inpainted_patch = inpaint_black_regions(cropped_patch)

    # Save inpainted patch
    cv2.imwrite(r"C:\Users\ariel\PycharmProjects\MLDM_Project\generations\patch_match_20_2/aligned_"+str(i).zfill(2)+".png", inpainted_patch[:500, :500])

print("All patches aligned, cropped, inpainted and saved.")

Processing image  D:/Datasets/MLDM Project/Data/0.20__2/cropped\cropped_01.jpg
Patch aligned using phase correlation
Processing image  D:/Datasets/MLDM Project/Data/0.20__2/cropped\cropped_02.jpg
Patch aligned using phase correlation
Processing image  D:/Datasets/MLDM Project/Data/0.20__2/cropped\cropped_03.jpg
Patch aligned using phase correlation
Processing image  D:/Datasets/MLDM Project/Data/0.20__2/cropped\cropped_04.jpg
Patch aligned using phase correlation
Processing image  D:/Datasets/MLDM Project/Data/0.20__2/cropped\cropped_05.jpg
Patch aligned using phase correlation
Processing image  D:/Datasets/MLDM Project/Data/0.20__2/cropped\cropped_06.jpg
Patch aligned using phase correlation
Processing image  D:/Datasets/MLDM Project/Data/0.20__2/cropped\cropped_07.jpg
Patch aligned using phase correlation
Processing image  D:/Datasets/MLDM Project/Data/0.20__2/cropped\cropped_08.jpg
Patch aligned using phase correlation
Processing image  D:/Datasets/MLDM Project/Data/0.20__2/cropped\